In [45]:
import numpy as np
import pandas as pd

from datetime import datetime

# from tqdm.notebook import tqdm

# import torch 
# import torch.nn as nn
# from torch.utils.data import DataLoader, Dataset
# import pytorch_lightning as pl

# np.random.seed(123)

# Loading Data

In [54]:
## load user ratings data
user_ratings = pd.read_csv("data/ratings.csv")
user_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [55]:
user_ratings.shape

(100836, 4)

In [56]:
user_ratings["timestamp"] = user_ratings["timestamp"].apply(lambda x: datetime.fromtimestamp(x))

In [57]:
user_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 11:45:03
1,1,3,4.0,2000-07-30 11:20:47
2,1,6,4.0,2000-07-30 11:37:04
3,1,47,5.0,2000-07-30 12:03:35
4,1,50,5.0,2000-07-30 11:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 14:53:22
100832,610,168248,5.0,2017-05-03 15:21:31
100833,610,168250,5.0,2017-05-08 12:50:47
100834,610,168252,5.0,2017-05-03 14:19:12


# Train Test Split

In [58]:
user_ratings['rank_latest'] = user_ratings.groupby(['userId'])['timestamp'] \
                                .rank(method = 'first',ascending=False)

train = user_ratings[user_ratings['rank_latest'] != 1]
test = user_ratings[user_ratings['rank_latest'] == 1]

# drop columns that we no Longer need 
train = train[['userId', 'movieId', 'rating']]
test = test[['userId','movieId','rating']]

# Data Preprocessing

In [59]:
## convert rating to binary to indicate watched 
train.loc[:, 'rating'] = 1
train.head()


,userId,movieId,rating
0,1,1,1
1,1,3,1
2,1,6,1
3,1,47,1
4,1,50,1


In [60]:
## add radom sample of unwatched movies to the dataset

# Get a list of all movie IDs
all_movieIds = user_ratings['movieId'].unique()

# Placeholfers that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each usesr has interaction with
user_item_set = set(zip(train['userId'],train['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

In [63]:
for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds)
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative
        
torch.tensor(users)
torch.tensor(items)
torch.tensor(labels)

  0%|          | 0/100226 [00:00<?, ?it/s]

tensor([1, 0, 0,  ..., 0, 0, 0])